# Lection8

## Deep Learning (CNN Computer Vision)



<img src="../img/cnn1.jpeg" width=600px>

[Tensorflow documentation](https://www.tensorflow.org/api_docs/python/tf)

[CNN explainer](https://poloclub.github.io/cnn-explainer/)

[Daniel Bourke github repo](https://github.com/mrdbourke/tensorflow-deep-learning/)





# Plan

1. Import TensorFlow
2. Data preparation
3. Modeling
4. Evaluation & Improving
5. Model save & load

In [ ]:
!wget https://storage.googleapis.com/ztm_tf_course/food_vision/pizza_steak.zip --no-check-certificate

In [ ]:
import zipfile

# Unzip the downloaded file
zip_ref = zipfile.ZipFile("pizza_steak.zip", "r")
zip_ref.extractall()
zip_ref.close()